In [ ]:
%matplotlib widget
import json
import logging
from pathlib import Path
from o3r_algo_utilities.calib import calibration
from xmlrpc.client import ServerProxy
logger = logging.getLogger(__name__)

In [ ]:
# configuration values
camPort = 2
ip = "192.168.0.69"
# XYZ coordinates [m] of the ABCD points
if True:
    # camera and target are mounted horizontally
    # A is upper left corner in the image and also in the world
    X_AB=1.290-0.065
    Z_AB=1.195
    X_CD=1.290-0.14
    Z_CD=0.60
    Y_AC=0.57
    Y_BD=-0.23
    if False:
        # 180 deg rotated
        X_AB, X_CD = X_CD, X_AB
        Z_AB, Z_CD = Z_CD, Z_AB
        Y_AC, Y_BD = Y_BD, Y_AC
        print("X_AB=",X_AB,"X_CD=",X_CD,"Z_AB=",Z_AB,"Z_CD=", Z_CD, "Y_AC=",Y_AC,"Y_BD=",Y_BD)
        
    # the size of the white border around the checkerboard [m]
    frame_size = 0.05
    # number of inner points on target
    target_width = 6
    target_height = 4
    pcc = "X_AB=%.3f,Z_AB=%.3f,X_CD=%.3f,Z_CD=%.3f,Y_AC=%.3f,Y_BD=%.3f" % (X_AB, Z_AB, X_CD, Z_CD, Y_AC, Y_BD)
else: 
    # camera and target are mounted horizontally
    # A is upper left corner in the image and lower left corner in the world
    target_width = 6
    target_height = 4
    frame_size = 0.05
    X_AC=1.227-0.057
    Z_AC=0.0
    X_BD=1.227
    Z_BD=0.798
    Y_AB=0.3
    Y_CD=-0.3
    pcc = "X_AC=%.3f,Z_AC=%.3f,X_BD=%.3f,Z_BD=%.3f,Y_AB=%.3f,Y_CD=%.3f" % (X_AC, Z_AC, X_BD, Z_BD, Y_AB, Y_CD)
    
source = "ifm3dpy://%s/port%d" % (ip, camPort)
# it's also possible to use a recording as source:
#source=r"adrec://C:\Projects\iCV-Algo\O3R\workspace\20210920_095332_calib2.h5"

# if you have good data for the camera position in [m], you can place it here. The 3 vector (tx,ty,tz) specifies
# the position of the head's reference point expressed in the robot coordinate system.
#fixed_translation = [0.25, -0.1, 0.5]
fixed_translation = None

# by default, the amplitude image is used to detect the corner points. As an alternative, you might try to use
# the reflectivity image instead.
# image_selection = "reflectivity"
image_selection = "amplitude"

args = dict(
    frame_size=frame_size,
    pattern_corner_coordinates=pcc,
    target_width=target_width,
    target_height=target_height,
    source=source,
    max_allowed_reconstruction_error=2,
    fixed_translation=fixed_translation,
    image_selection=image_selection,
)
trans, rot = calibration.calib(**args)

In [ ]:
# Push the calibrated values to the device. The values are saved via saveInit()
if ip is not None:
    prx = ServerProxy("http://%s/api/rpc/v1/com.ifm.efector/" % ip)
    oldCalib = json.loads(prx.get(["/ports/port%d/processing/extrinsicHeadToUser"%camPort]))
    logger.info("oldCalib=%s", oldCalib["ports"]["port%d"%camPort]["processing"]["extrinsicHeadToUser"])
    newCalib = {
        "ports":
        {
            "port%d"%camPort:
            {
                # make sure to save the mode together with the extrinsic calibration to avoid issues when changing the mode
                "mode": "cyclic_4m_2m_4m_2m", 
                "processing": 
                {
                    "extrinsicHeadToUser":dict(
                        transX = trans[0], transY = trans[1], transZ = trans[2],
                        rotX = rot[0], rotY = rot[1], rotZ = rot[2],
                    )
                }
            }
        }
    }
    prx.set(json.dumps(newCalib))
    newCalib = json.loads(prx.get(["/ports/port%d/processing/extrinsicHeadToUser"%camPort]))
    logger.info("newCalib=%s", newCalib["ports"]["port%d"%camPort]["processing"]["extrinsicHeadToUser"])
    prx.saveInit()

In [ ]:
# IMU calibration, measure position and use table in word document for rotations
# please also take a photo of the VPU mounted on the vehicle where the orientation can be double-checked afterwards
transZ=0.795+0.03
transX=0.045
transY=-0.04
rotX=3.1415
rotY=0
rotZ=-3.1415/2
newCalib = {
    "ports":
    {
        "port6":
        {
            # make sure to save the mode together with the extrinsic calibration to avoid issues when changing the mode
            # also ensures that the correct mode is being used
            "mode": "experimental_ods",
            "processing": 
            {
                "extrinsicVPUToUser":dict(
                    transX = transX, transY = transY, transZ = transZ,
                    rotX = rotX, rotY = rotY, rotZ = rotZ,
                )
            }
        }
    }
}
prx.set(json.dumps(newCalib))
prx.saveInit()
calib = prx.get(["/ports/port6/processing/extrinsicVPUToUser"])
logger.info("newCalib=%s", calib)

In [ ]:
prx = ServerProxy("http://%s/api/rpc/v1/com.ifm.efector/" % ip)
print(json.dumps(json.loads(prx.get([""])), indent=2))
# things which are important, please check this manually:
# 1. The modes of the 3D ports must be set to "cyclic_4m_2m_4m_2m"
# 2. The mode of port6 must be "experimental_ods"
# 3. The extrinsic calibration of the 3D ports must match with what has been determined during calibration
# 4. The extrinsic calibration of port6 must match with what has been determined during calibration